In [1]:
#このセルでは、ひとまずSuperconのデータを使ってcrabnetを適当に試してみる

In [1]:
import pandas as pd
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

In [4]:
#まずsuperconのデータを適当に分析する

SC_data = pd.read_csv("/workspaces/mat2vec_codespace/main_code/20240322_Supercon_data_allstr_handedited.csv", encoding='utf-8')
missing_values = SC_data.isnull().sum()
print(missing_values)

/workspaces/mat2vec_codespace/.venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (0,6,8,10,12,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,58,59,61,62,63,64,65,66,67,68,69,70,71,72,73,76,77,78,79,80,81,82,84,86,87,88,89,90,91,92,94,95,96,97,98,99,100,101,103,104,105,106,107,108,109,110,112,113,114,115,117,119,120,121,122,124,125,126,127,128,129,130,131,132,133,134,136,137,139,140,141,142,143,144,145,146,147,148,149,150,151,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,174,175,176,177,179,180,181,182,184,190) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


data number                                                         0
reference number                                                    0
comment                                                         18287
common formula of materials                                      1522
chemical formula                                                    0
element name of materials                                           0
composition of MA1                                                  0
element name of materials.1                                       440
composition of MA2                                                455
element name of materials.2                                      6704
composition of MA3                                               6721
element name of materials.3                                     17533
composition of MA4                                              17565
element name of materials.4                                     27678
composition of MA5  

In [5]:
crab_input_data = SC_data.loc[:, ["chemical formula", "Tc (of this sample) recommended", "oxygen", "common formula of oxygen", "measured value of Oxygen content", "title of reference"]].rename(columns={"chemical formula": "formula", "Tc (of this sample) recommended": "Tc"})

In [6]:
crab_input_data = crab_input_data[crab_input_data["Tc"].notna()]
crab_input_data = crab_input_data.iloc[1:, :]

In [8]:
#現状判明していることとして、酸素数関連のデータに問題がいろいろとあるので
#酸素がある物質については酸素の具体的な量が乗っているデータは組成式にそれを組み込んで
#使ってないデータは消す方針でいったん動く
#まず酸化物とそうでないものを分ける

In [7]:
NonOxide_SC_data = crab_input_data[crab_input_data["oxygen"].isna()].drop(columns=["oxygen", "common formula of oxygen", "measured value of Oxygen content"])
Oxide_SC_data = crab_input_data[crab_input_data["oxygen"].notna()]

In [8]:
#measured valueがある場合
measured_value_of_Oxygen_index = Oxide_SC_data.loc[:, "measured value of Oxygen content"].notna()
Oxide_SC_data.loc[measured_value_of_Oxygen_index, "value of oxygen"] = Oxide_SC_data.loc[measured_value_of_Oxygen_index, "measured value of Oxygen content"]


/workspaces/mat2vec_codespace/.venv/lib/python3.7/site-packages/pandas/core/indexing.py:1681: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = empty_value
/workspaces/mat2vec_codespace/.venv/lib/python3.7/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [9]:
#measured valueがないがcommon formulaに数字がある場合
common_value_of_Oxygen_index = Oxide_SC_data.loc[:, "measured value of Oxygen content"].isna() & Oxide_SC_data.loc[:, "common formula of oxygen"].str.isdigit()
Oxide_SC_data.loc[common_value_of_Oxygen_index, "value of oxygen"] = Oxide_SC_data.loc[common_value_of_Oxygen_index, "common formula of oxygen"]

In [10]:
final_Oxide_SC_data = Oxide_SC_data.drop(columns=["oxygen", "common formula of oxygen", "measured value of Oxygen content"])
final_Oxide_SC_data = final_Oxide_SC_data[final_Oxide_SC_data["value of oxygen"].notna()]

In [11]:
new_SC_data = pd.concat([final_Oxide_SC_data, NonOxide_SC_data], axis = 0).drop(columns=["value of oxygen"]).reset_index(drop = True)

In [12]:
new_SC_data["Tc"] = new_SC_data["Tc"].astype(float)

In [13]:
pd.set_option('display.max_colwidth', 500)

In [20]:
display(final_SC_data.head(50))

,formula,Tc
3348,Hg0.75Ba2.07Ca2.07Cu3.11O8.208,135.8
3347,Hg0.75Ba2.07Ca2.07Cu3.11O8.187,135.4
4462,Hg1Ba2Ca2Cu3O8.27,135.0
4837,Hg1Ba2Ca2Cu3O8.29,135.0
4836,Hg1Ba2Ca2Cu3O8.29,135.0
3345,Hg0.75Ba2.07Ca2.07Cu3.11O8.15,134.7
3346,Hg0.75Ba2.07Ca2.07Cu3.11O8.16,134.2
2278,Hg0.8Mo0.2Ba2Ca2Cu3O8.4,134.0
2280,Hg0.6Ti0.4Ba2Ca2Cu3O8.4,134.0
5111,Hg1Ba2Ca2Cu3O8.35,134.0


In [16]:
new_SC_data = new_SC_data.sort_values("Tc", ascending = False)

In [17]:
check_words = ["pressure", "gpa", "mbar"]

In [18]:
high_pressure_data = new_SC_data["title of reference"].astype(str).apply(lambda x: any(word.lower() in x.lower() for word in check_words))
not_high_pressure_data = ~high_pressure_data

In [19]:
final_SC_data = new_SC_data[not_high_pressure_data].iloc[5:, :].drop(columns = ["title of reference"])

In [21]:
final_SC_data.to_csv("/workspaces/mat2vec_codespace/main_code/final_SC_X_data.csv", index = False)